
# Incorporating OLI Calculations with WaterTAP

Presented by Paul Vecchiarelli (NREL)

## Rationale

 - Simulations for realistic water sources are mathematically complex: 
 > $ Interactions \ge Cations * Anions$
 - OLI improves WaterTAP approximations and offloads computational resources

## OLI Cloud Inputs


 - Solute concentrations
 
 - Login credentials
 
 - A chemistry (*.dbs) file
     - contains state variables (temperature, pressure, solutes), phases, etc.
     
 - Input parameters for OLI call

## Basic Workflow

 - We will get scaling tendency computations from OLI Cloud via the following steps:
 
<img src="workflow.svg" alt_text="WaterTAP helper methods">

     

In [ ]:
from pyomo.environ import units as pyunits

from numpy import linspace

from watertap.tools.oli_api.util.state_block_helper_functions import create_state_block, extract_state_vars

from watertap.tools.oli_api.credentials import CredentialManager
from watertap.tools.oli_api.client import OLIApi

from watertap.tools.oli_api.core.water_analysis import WaterAnalysis

# 1. Specify State Variables

- We will make a state block from a dict:

In [ ]:
source_water = {
        'temperature': 298.15,
        'pressure': 101325,
        'components': {
            'Cl_-': 870,
            'Na_+': 739,
            'SO4_2-': 1011,
            'Mg_2+': 90,
            'Ca_2+': 258,
            'K_+': 9,
            'HCO3_-': 385,
            },
        'units': {
            'temperature': pyunits.K,
            'pressure': pyunits.Pa,
            'components': pyunits.mg/pyunits.L}
        }

- This data can be used to construct inputs to OLI

# 2. Get Survey Parameters

 - We will generate a survey investigating different brine concentrations:

In [ ]:
m = create_state_block(source_water)
state_block = m.fs.stream[0]
conc_var = state_block.conc_mass_phase_comp
state_vars = extract_state_vars(state_block, conc_var, source_water["units"])

In [ ]:
print(f"\nState Vars:\n{state_vars}")

In [ ]:
survey_conditions = {"SO4_2-": linspace(0, 1e3, 10)}

# 3. Create Input List

 - We updated our workflow from manual entry...

In [ ]:
# don't run this cell, it's just an example code
"""
def create_input_dict(self, AllowSolidsToForm=False):
    
    ...
    
    tmp_list.append(
        {
            "group": "Calculation Options",
            "name": "CalcType",
            "value": "EquilCalcOnly",
        }
    )
    tmp_list.append(
        {
            "group": "Calculation Options",
            "name": "AllowSolidsToForm",
            "value": AllowSolidsToForm,
        }
    )
    
    ...
"""

 - ...to an automated process that uses prebuilt, editable dictionaries to configure OLI calls

In [ ]:
water_analysis = WaterAnalysis(state_vars=state_vars,
                               survey_conditions=survey_conditions)

water_analysis.oli_input_dict["AllowSolidsToForm"] = True
props = {"scalingIndex": False,
         "prescalingTendencies": True,
         "prescalingTendenciesRigorous": True,
         "scalingTendencies": True,
         "MBGComposition" : False,
         "materialBalanceGroup": False} 
water_analysis.oli_optional_properties.update(props)

# 4. Provide Login Details

- The following code demonstrates an OLI Cloud login:

In [ ]:
# replace dummy credentials with your own

credentials = {"username": "dummy@dummy.edu",
               "password": "dummy_pass",
               "root_url": "https://dummy_root.com",
               "auth_url": "https://dummy_url.com/dummy",
              }

credential_manager = CredentialManager(**credentials)

key = credential_manager.encryption_key

 - An encryption key is provided at first login for future usage:

In [ ]:
credential_manager = CredentialManager(encryption_key=key)

# 5. Create *.dbs File and 6. Get Raw Output

 - In this example we will simulate brine concentration

In [ ]:
# will take 20-30 seconds to run

survey = water_analysis.build_composition_survey(survey_conditions)

solute_list = source_water["components"]
phases = ["liquid1", "solid"]

with OLIApi(credential_manager) as oliapi:

    dbs_file_id = oliapi.get_dbs_file_id(chemistry_source=solute_list,
                                         phases=phases,
                                         model_name="remote_file_from_dict")
    
    water_analysis.run(oliapi=oliapi,
                       dbs_file_id=dbs_file_id)

# 7. Extract Filtered Output

 - OLI's output is robust, so WaterTAP enables printing selected results:

In [ ]:
print("\nPhase Properties:")

extracted_properties = water_analysis.extract_basic_properties(phase="liquid1",
                                                               properties=["osmoticPressure", "ph"])
print(extracted_properties)

In [ ]:
print("\nScaling Tendencies:")

extracted_scaling_tendencies = water_analysis.extract_scaling_tendencies(scalants=["CACO3", "CASO4.2H2O"])    
print(extracted_scaling_tendencies)

# Conclusions

OLI can be used to strengthen WaterTAP in several ways:

1. Modeling physico-chemical properties of a system
2. Investigating pre- and equilibrium scaling tendency
3. Surveying various conditions within a model

**Features will be implemented in a future WaterTAP release.**

Contact: Paul Vecchiarelli (paul.vecchiarelli@nrel.gov)